In [17]:
import pandas as pd
df = pd.read_csv('hmnist_64_64_RGB.csv')

In [18]:
df.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel12279,pixel12280,pixel12281,pixel12282,pixel12283,pixel12284,pixel12285,pixel12286,pixel12287,label
0,191,152,194,191,153,195,192,149,192,194,...,185,154,180,186,156,184,182,152,173,2
1,24,13,23,24,14,28,37,24,46,61,...,26,15,29,27,15,28,24,13,25,2
2,185,129,140,192,136,151,198,142,156,198,...,152,120,138,144,113,123,115,81,84,2
3,24,11,19,36,19,30,64,38,50,91,...,27,13,16,25,12,16,24,9,14,2
4,138,94,117,158,113,138,178,133,161,191,...,140,106,118,108,77,92,67,40,55,2


In [19]:
df.shape

(10015, 12289)

In [20]:
x = df.drop("label", axis=1).values
y = df["label"].values

In [21]:
x.shape, y.shape

((10015, 12288), (10015,))

In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)

In [23]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8012, 12288), (2003, 12288), (8012,), (2003,))

In [24]:
x_train = x_train.reshape(x_train.shape[0], *(64, 64, 3))
x_test = x_test.reshape(x_test.shape[0], *(64, 64, 3))

In [25]:
x_train.shape, x_test.shape

((8012, 64, 64, 3), (2003, 64, 64, 3))

In [26]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [27]:
y_train.shape, y_test.shape

((8012, 7), (2003, 7))

In [28]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.applications import ResNet50
from keras import regularizers
import numpy as np

model = Sequential()
num_labels = 7

base_model = ResNet50(include_top=False,input_shape=(64,64,3),pooling = 'avg', weights="imagenet")
model = Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.02)))
model.add(Dropout(0.5))
model.add(Dense(num_labels, activation = 'softmax',kernel_regularizer=regularizers.l2(0.02)))

for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-22:]:
    layer.trainable = True

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               262272    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 7)                 903       
                                                                 
Total params: 23,850,887
Trainable params: 9,194,503
Non-trainable params: 14,656,384
_________________________________________________________________


In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_accuracy',patience=5,mode='max',restore_best_weights=True)

In [31]:
hist = model.fit(x_train,y_train,epochs=5,batch_size=32,verbose=1,validation_split=0.2,callbacks=es)

Epoch 1/5
201/201 [==============================] - 269s 1s/step - loss: 3.2002 - accuracy: 0.6767 - val_loss: 1.8418 - val_accuracy: 0.7293
Epoch 2/5
201/201 [==============================] - 263s 1s/step - loss: 1.4201 - accuracy: 0.7284 - val_loss: 1.1383 - val_accuracy: 0.7642
Epoch 3/5
201/201 [==============================] - 291s 1s/step - loss: 1.0188 - accuracy: 0.7535 - val_loss: 0.9837 - val_accuracy: 0.7517
Epoch 4/5
201/201 [==============================] - 307s 2s/step - loss: 0.8407 - accuracy: 0.7786 - val_loss: 0.9300 - val_accuracy: 0.7330
Epoch 5/5
201/201 [==============================] - 311s 2s/step - loss: 0.7322 - accuracy: 0.8028 - val_loss: 0.8999 - val_accuracy: 0.7629


In [32]:
import numpy as np
x_test=np.array(x_test).reshape(-1,64,64,3)
score=model.evaluate(x_test,y_test)
print("loss: ",score[0])
print("Accuracy: ",score[1])

63/63 [==============================] - 18s 292ms/step - loss: 0.9450 - accuracy: 0.7294
loss:  0.9449655413627625
Accuracy:  0.7294058799743652


In [33]:
classes={0:('akiec', 'actinic keratoses and intraepithelial carcinomae'),
         1:('bcc' , 'basal cell carcinoma'),
         2:('bkl', 'benign keratosis-like lesions'),
         3:('df', 'dermatofibroma'),
         4:('nv', ' melanocytic nevi'),
         5:('vasc', ' pyogenic granulomas and hemorrhage'),
         6:('mel', 'melanoma'),}

In [34]:
import PIL
image=PIL.Image.open('test/ISIC_0010512.jpg')
image=image.resize((64,64))
img=x_test[1]
img=np.array(image).reshape(-1,64,64,3)
result=model.predict(img)
print(result[0])
result=result.tolist()
max_prob=max(result[0])
class_ind=result[0].index(max_prob)
print(classes[class_ind])

1/1 [==============================] - 1s 1s/step
[0.01441294 0.01367702 0.72692245 0.0043053  0.07769188 0.00176272
 0.16122766]
('bkl', 'benign keratosis-like lesions')


In [35]:
model.save("ResNet.h5")

In [36]:
from keras.models import load_model
import tensorflow as tf

new_cnn = load_model("ResNet.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(new_cnn)
tflite_model = converter.convert()

with open("resnet.tflite",'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\DEEPKH~1\AppData\Local\Temp\tmp5uu494ko\assets


INFO:tensorflow:Assets written to: C:\Users\DEEPKH~1\AppData\Local\Temp\tmp5uu494ko\assets
